# 数据预处理

In [1]:
import numpy as np                                        # 导入numpy库并简写为np
from sklearn import datasets                              # 导入datasets模块，用于加载鸢尾花的数据集

iris_dataset = datasets.load_iris()                       # 加载鸢尾花的数据集，并存在iris_dataset

In [2]:
X = iris_dataset.data[:100, :].astype(np.float32)         # 选取iris_dataset的data的前100个数据，将其数据类型转换为float32，并储存在X中
X_feature_names = iris_dataset.feature_names              # 将iris_dataset的特征名称储存在X_feature_names中
y = iris_dataset.target[:100].astype(int)                 # 选取iris_dataset的target的前100个数据，将其数据类型转换为int，并储存在y中
y_target_names = iris_dataset.target_names[:2]            # 选取iris_dataset的target_names的前2个数据，并储存在y_target_names中


In [3]:
alpha = X[:, :3] * X[:, 1:]           # 每一个样本中，利用相邻两个特征值计算出一个参数，即每一个样本会多出3个参数（因为有4个特征值），并储存在alpha中
X = np.append(X, alpha, axis=1)       # 在axis=1的维度上，将alpha的数据值添加到X的特征值中

print(X.shape)                        # 打印此时X的样本的数据维度

(100, 7)


In [4]:
def feature_normalize(data):
    mu = np.mean(data,axis=0)
    std = np.std(data,axis=0)
    return (data - mu)/std
X = feature_normalize(X)

In [5]:
from sklearn.model_selection import train_test_split                                                   # 导入train_test_split函数，用于对数据集进行划分

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, shuffle=True) # 将数据集划分为训练集和测试集
print(X_train.shape)                                                                                   # 打印训练集中样本的数据类型
print(X_test.shape)
print(y_test)


(80, 7)
(20, 7)
[0 1 0 1 1 1 0 1 1 1 1 1 1 0 0 0 0 0 0 0]


In [6]:
# sigmoid function
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

## 数据归一化

## 3.3 将程序提交到量子计算云平台(同QCIS流程)<a id="jump_3_3"></a>

将编译出的QCIS线路提交到量子计算云平台。

In [7]:
from pyqcisim.simulator import *
from numpy import pi

def param_qcis_circuit(params: list):
    assert len(params) == 15 
    alpha0 = params[0]
    alpha1 = params[1]
    alpha2 = params[2]
    alpha3 = params[3]
    alpha4 = params[4]
    alpha5 = params[5]
    alpha6 = params[6]
    theta0 = params[7]
    theta1 = params[8]
    theta2 = params[9]
    theta3 = params[10]
    theta4 = params[11]
    theta5 = params[12]
    theta6 = params[13]
    theta7 = params[14]
    qcis_circuit = f'''
H Q1
H Q2
H Q3
H Q4

RZ Q1 {alpha0}
RZ Q2 {alpha1}
RZ Q3 {alpha2}
RZ Q4 {alpha3}

CZ Q1 Q2
RZ Q2 {alpha4}
CZ Q1 Q2

CZ Q2 Q3
RZ Q3 {alpha5}
CZ Q2 Q3

CZ Q3 Q4
RZ Q4 {alpha6}
CZ Q3 Q4

RY Q1 {theta0}
RY Q2 {theta1}
RY Q3 {theta2}
RY Q4 {theta3}

CZ Q1 Q2
CZ Q2 Q3
CZ Q3 Q4

RY Q1 {theta4}
RY Q2 {theta5}
RY Q3 {theta6}
RY Q4 {theta7}

M Q3
M Q4 
   '''
    return qcis_circuit
    

## Todo:
1. 梯度计算
$\frac{\partial L}{\partial \theta_i} = \frac{<B>^+_i - <B>^-_i}{2}$
2. $\theta$ 的更新
$\theta_{j+1} = \theta_j - \alpha \frac{\partial L}{\partial \theta_j}$
3. loss定义
$loss = -\sum_{i=1}^m y_i\log(\hat{y})$

In [48]:
import random
# train
pyqcisim = PyQCISim()
eta = 0.1
loss = 0
theta = [1 for i in range(8)]
epoch = 10
batch_size = 2

def loss_f(forward_res_ls: np.array, y: np.array):
    # 计算loss 交叉熵
    sum_loss = 0
    n = forward_res_ls.shape[0]
    loss = -np.sum(y * np.log(forward_res_ls)) / n
    # for i in range(l):
    #     sum_loss += y[i] * np.log(forward_res_ls[i]) + (1 - y[i]) * np.log(1 - forward_res_ls[i])
    print("loss: ", loss)   
    return loss

# test loss_f

print("loss: ", loss_f(np.array([0.5, 0.5, 0.5, 0.5, 0.5]), np.array([1, 1, 1, 1, 1])))

log_file = open("log.txt", "w")

for e in range(epoch):
    forward_res_ls = np.zeros((batch_size, 1))
    y = np.zeros((batch_size, 1))
    gradient_ls = np.zeros((batch_size, len(theta) ))
    print("epoch: ", e)

    for j in range(batch_size):
        idx = random.randint(0, 80)
        alpha = X_train[idx].tolist()
        # 拼接参数列表
        params = alpha + theta
        forward_res = 0
        gradient_ls_batch = []
        for i in range(8):
            params[i+7] += pi/2

            qcis_circuit = param_qcis_circuit(params)
            pyqcisim.compile(qcis_circuit)
            msmt_result = pyqcisim.simulate("one_shot", num_shots=1000) # num_shots会影响计算速度，原始采用12000，这里为了快速验证，采用1000
            msmt_result = msmt_result[1]
            # print("msmt_result: ", msmt_result)
            prob_1 = msmt_result['10']+msmt_result['11']
            B_plus =prob_1
            # B_plus = sigmoid(prob_1)
            # # query_id=account.run_experiment(exp_id=exp_id, num_shots=12000)#num_shots在上述submit_job中采用了默认值12000。

            # # if query_id:
            # #     result=account.query_experiment(query_id, max_wait_time=360000)
            # #     #最大等待时间单位为秒，不传递时默认为30秒。因量子程序的执行会有排队的情况，而量子计算机本身有自动校准的时间，如果想跑全自动的程序，等待时间最好大于两者。
            # #     prob_1 = account.readout_data_to_state_probabilities_whole(result)['10']+account.readout_data_to_state_probabilities_whole(result)['11']
            # #     #后继数据应用实现。
            # #     B_plus = sigmoid(prob_1)
            if i == 7:
                forward_res = B_plus / 1000
                forward_res_ls[j] = forward_res
                y[j] = y_train[idx]

            params[i+7] -= pi
            qcis_circuit = param_qcis_circuit(params)
            pyqcisim.compile(qcis_circuit)
            msmt_result = pyqcisim.simulate("one_shot", num_shots=1000) # num_shots会影响计算速度，原始采用12000，这里为了快速验证，采用1000
            # print("msmt_result: ", msmt_result)
            msmt_result = msmt_result[1]
            prob_2 = msmt_result['10']+msmt_result['11']
            # B_minus = sigmoid(prob_2)
            B_minus = prob_2

            # qcis_circuit_with_params = account.assign_parameters(circuit=qcis_circuit, parameters=['alpha0', 'alpha1', 'alpha2', 'alpha3', 'alpha4', 'alpha5', 'alpha6', 'theta0', 'theta1', 'theta2', 'theta3', 'theta4', 'theta5', 'theta6', 'theta7'], values=params)
            # query_id=account.run_experiment(exp_id=exp_id, num_shots=12000)#num_shots在上述submit_job中采用了默认值12000。
            # if query_id:
            #     result=account.query_experiment(query_id, max_wait_time=360000)
            #     #最大等待时间单位为秒，不传递时默认为30秒。因量子程序的执行会有排队的情况，而量子计算机本身有自动校准的时间，如果想跑全自动的程序，等待时间最好大于两者。
            #     prob_1 = account.readout_data_to_state_probabilities_whole(result)['10']+account.readout_data_to_state_probabilities_whole(result)['11']
            #     #后继数据应用实现  
            #     B_minus = sigmoid(prob_1)
            gradient = (prob_1 - prob_2)/2
            gradient_ls[j][i] = gradient
    gradient = np.mean(gradient_ls, axis=0) # 形状：（len(theta)，）
    assert len(gradient) == len(theta)
    print("gradient: ", gradient)
    theta = np.array(theta) - eta * gradient
    theta = theta.tolist()
    loss = loss_f(forward_res_ls, y_train)
    log_file.write("epoch: " + str(e) + " loss: " + str(loss) + "\n")
log_file.close()
print("params:", alpha + theta)


loss:  0.6931471805599453
loss:  0.6931471805599453
epoch:  0
num_qubits:  2
num_qubits:  2
num_qubits:  2
num_qubits:  2
num_qubits:  2
num_qubits:  2
num_qubits:  2
num_qubits:  2
num_qubits:  2
num_qubits:  2
num_qubits:  2
num_qubits:  2
num_qubits:  2
num_qubits:  2
num_qubits:  2
num_qubits:  2
num_qubits:  2
num_qubits:  2
num_qubits:  2
num_qubits:  2
num_qubits:  2
num_qubits:  2
num_qubits:  2
num_qubits:  2
num_qubits:  2
num_qubits:  2
num_qubits:  2
num_qubits:  2
num_qubits:  2
num_qubits:  2
num_qubits:  2
num_qubits:  2
gradient:  [ -3.     4.25 -51.5  -42.     6.75  18.75  21.75 104.5 ]
loss:  21.505612034341596
epoch:  1
num_qubits:  2
num_qubits:  2
num_qubits:  2
num_qubits:  2
num_qubits:  2
num_qubits:  2
num_qubits:  2
num_qubits:  2
num_qubits:  2
num_qubits:  2
num_qubits:  2
num_qubits:  2
num_qubits:  2
num_qubits:  2
num_qubits:  2
num_qubits:  2
num_qubits:  2
num_qubits:  2
num_qubits:  2
num_qubits:  2
num_qubits:  2
num_qubits:  2
num_qubits:  2
num_qubi

In [49]:
theta

[0.5749999999999998,
 -1.6500000000000006,
 15.450000000000001,
 6.475000000000001,
 -2.1750000000000007,
 -1.775,
 -3.1750000000000007,
 -46.15000000000001]

In [50]:
correct = 0
# for i in range(8):
#     params[i+7] += pi/2

pred=  []
for j in range(20):
    alpha = X_test[j].tolist()
    # params = alpha + [1,1,1,1,1,1,1,1]
    params = alpha + theta
    qcis_circuit = param_qcis_circuit(params)
    pyqcisim.compile(qcis_circuit)
    msmt_result = pyqcisim.simulate("one_shot", num_shots=1000) # num_shots会影响计算速度，原始采用12000，这里为了快速验证，采用1000
    msmt_result = msmt_result[1]
    print("msmt_result: ", msmt_result)
    prob_1 = msmt_result['10']+msmt_result['11']
    prob_2 = msmt_result['00']+msmt_result['01']
    if prob_1 > prob_2 and y_test[j] == 0:
        correct += 1
    if prob_1 < prob_2 and y_test[j] == 1:
        correct += 1
    if prob_1 > prob_2:
        pred.append(0)
    else:
        pred.append(1)
    
print(correct/20)

num_qubits:  2
msmt_result:  {'00': 260, '01': 249, '10': 258, '11': 233}
num_qubits:  2
msmt_result:  {'00': 302, '01': 45, '10': 76, '11': 577}
num_qubits:  2
msmt_result:  {'00': 241, '01': 184, '10': 150, '11': 425}
num_qubits:  2
msmt_result:  {'00': 252, '01': 231, '10': 170, '11': 347}
num_qubits:  2
msmt_result:  {'00': 300, '01': 153, '10': 117, '11': 430}
num_qubits:  2
msmt_result:  {'00': 112, '01': 332, '10': 527, '11': 29}
num_qubits:  2
msmt_result:  {'00': 250, '01': 265, '10': 210, '11': 275}
num_qubits:  2
msmt_result:  {'00': 197, '01': 298, '10': 182, '11': 323}
num_qubits:  2
msmt_result:  {'00': 332, '01': 62, '10': 78, '11': 528}
num_qubits:  2
msmt_result:  {'00': 203, '01': 309, '10': 240, '11': 248}
num_qubits:  2
msmt_result:  {'00': 307, '01': 168, '10': 266, '11': 259}
num_qubits:  2
msmt_result:  {'00': 206, '01': 305, '10': 300, '11': 189}
num_qubits:  2
msmt_result:  {'00': 325, '01': 56, '10': 80, '11': 539}
num_qubits:  2
msmt_result:  {'00': 299, '01'

In [44]:
print(pred)
y = y_test[:20]
print(y)
print(np.array(pred))
# 使用BCELoss计算loss
def BCEloss(pred: np.array, y: np.array):
    sum_loss = 0
    for i in range(len(pred)):
        if pred[i] == 0:
            p = 1 - pred[i]
        elif pred[i] == 1:
            p = pred[i]
        sum_loss += -y[i] * np.log(p + 1e15) - (1 - y[i]) * np.log(1 - p + 1e15)
    print("loss: ", sum_loss/len(pred))
    return sum_loss/len(pred)


[1]
[0 1 0 1 1 1 0 1 1 1 1 1 1 0 0 0 0 0 0 0]
[1]
loss:  -34.538776394910684


-34.538776394910684